#프로젝트: KoChatGPT 업그레이드

### 목표
아래 제시된 또는 그밖의 방법으로 커스텀 ChatGPT를 개발한다.

- 기존 KoChatGPT 모델에 사용한 데이터셋 정제

- SFT와 RM 모델에 쓰이는 사람 피드백(Human Feedback) 데이터셋을 다른 benchmark 데이터셋으로 대체

- 디코딩 성능 개선을 위한 하이퍼파라미터 조정

- 정량 메트릭 설정하여 모델 선택

- 지도 조정(Instruction Tuning) 및 프롬프팅 기법 적용

- 더 큰 기반 모델(foundation model) 사용

- 저순위 적응(LoRA, Low Rank Adaptaion), 지도 조정, 보상 순위(reward ranking) 알고리즘 도입

### 주의사항

기존 소스코드는 빠른 실습을 위해 원본 코드를 일부 수정하였으므로, 모델을 커스터마이징 과정에서 필요한 경우 "colossalai_ChatGPT_230319" 폴더 내의 원본 스크립트들을 참고할 것

### 실행 예시
#### 데이터셋 정제
- `data_kochatgpt` 디렉토리 내의 `kochatgpt_1_SFT.jsonl`, `kochatgpt_1_RM.jsonl`,`kochatgpt_1_PPO.jsonl` 파일을 분석(EDA)하여 정제
- 정제 후 데이터 증강 기법 적용하여 줄어든 데이터 크기를 유지 또는 증량

#### 새 데이터셋 추가
- 공개댓글을 활용해 순위(ranking) 데이터셋 구축한 앤스로픽(Anthropic) RLHF 구현을 참고하여, 동일 프롬프트에 대한 여러 반응(댓글)을 크롤링하여 차등적 보상을 적용하는 지시/랭킹 데이터셋 구축

-  KorQuad 등 한국어 이해 벤치마크를 활용한 고품질 데이터셋을 확보해 KoGPT-2의 저품질 데이터셋과 페어링

- 다양한 데이터 증강 시도하여 모델 재학습

#### 기반모델 변경
- 허깅페이스가 제공하는 경량화/최적화 라이브러리 적용하여 튜닝 (예: skt/ko-gpt-trinity 모델)
- 메모리 초과(OOM) 문제 예방을 위해 적절한 훈련 인자 조정이 필요
- 기반 모델 교체 후 generator 함수를 수정하여 추론 결과 제시


### 평가
- 새로운 모델 학습 후 BLEU, ROUGE등 정량 지표 활용하여 추론 성능 비교

# 환경 설정

In [ ]:
from google.colab import drive
import os

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 코랩 환경에서 실행을 위핸 패키지 업데이트 (실행 후 세션 재시작)
chatgpt_package_path = '/content/drive/MyDrive/Data/KoChatGPT/colossalai_ChatGPT_230319'

!pip install {chatgpt_package_path} datasets loralib accelerate
!pip install colossalai==0.2.8

os.kill(os.getpid(), 9)

Mounted at /content/drive
Processing ./drive/MyDrive/Data/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.1/728.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.7 MB/s eta 0:00:00
  Created wheel for colossalai: filename=colossalai-0.2.8-py3-none-any.whl size=930922 sha256=1da1ec8a108777586fc0457651196d465e4542366ea9904ee547e0901a693467
  Stored in directory: /root/.cache/pip/wheels/91/4d/77/9e915ab39a58f922cb9c4db126e8a082b56131e81339d64fcb
Successfully built colossalai
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
  Att

In [2]:
## 버전 확인

import torch

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
!pip list | grep transformers # transformers 4.28.0

Torch version:1.13.1+cu117
Cuda version: 11.7
transformers                     4.35.2


# 모델 및 토크나이저 불러오기

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [4]:
import pandas as pd
import numpy

# 토크나이저 작동 확인
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()
tokens = tokenizer(input_txt).tokens()

df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,...,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,...,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [5]:
# 디코딩 모델 작동 확인 (탐욕적 탐색)
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=128,
                               do_sample=False) # 예상 토큰 인덱스 벡터

print(tokenizer.decode(output_greedy[0])) # 토크나이저의 디코더로 출력

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


In [6]:
# 모델 작동 확인 (빔 서치; n-gram 페널티 부여)
output_beam = model.generate(input_ids, max_length=128, num_beams=10,
                             no_repeat_ngram_size=2, do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


In [7]:
# 모델 작동 확인 (빔 서치; n-gram 페널티 부여, top_k 샘플링 기법 추가)
output_beam = model.generate(input_ids, max_length=128, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.</d> 올해 수능에서 국어 1등급 커트라인은 국어 A·B등급의 평균 점수인 5.06점이고, 수학 가형은 국어가 5.47점, 수학 나형은 5.27점으로 나타났다.
1등급과 2등급을 합치면 국어 A, B등급은 국어, 수학B, 영어, 사회/과학, 과학탐구가 당락을 가를 것으로 보인다.
지난해 수준과 비슷하거나 약간 상승한 수치다.
평가원 관계자는 "수능은 지난 6월 모의평가와 비교해 다소 어렵게 출


In [8]:
# 모델 작동 확인 (빔 서치; n-gram 페널티 부여, top_p 샘플링 기법 추가)
output_beam = model.generate(input_ids, max_length=128, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지. 그게 무슨 소리냐?"
그녀는 고개를 갸웃거렸다.
"아니, 그건 그렇고."
"어떻게 된 일입니까."
그녀가 고개를 끄덕였다.
"무슨 말씀을 하시는 겁니까, 당신. 당신이 그걸 어떻게 알겠습니까."
그러자 그녀는 고개를 저었다.
"당신은 그 말을 믿지 않습니다. 당신은 당신에게 그런 말을 한 적이 없습니다."
이윽고 그녀는 한숨을 내쉬었다.
그녀의 눈에는 눈물이


# 데이터셋 불러오기

In [9]:
# 지도 미조정(Supervised Fine-Tuning) 훈련 데이터
import json
from os.path import join as path_join

repo_path = '/content/drive/MyDrive/Data/KoChatGPT/'
sft_jsonl = path_join(repo_path, 'data_kochatgpt/kochatgpt_1_SFT.jsonl')
with open(sft_jsonl, "r", encoding='utf-8-sig') as json_file:
  list_data_dict = json.load(json_file)

print(f'데이터셋 길이: {len(list_data_dict)}')
list_data_dict[:3]

데이터셋 길이: 12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

In [10]:
# 보상 모델(Reward Model) 훈련 데이터
rm_jsonl = path_join(repo_path, 'data_kochatgpt/kochatgpt_2_RM.jsonl')
with open(rm_jsonl, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [11]:
# 인접향 최적화(Proximal Policy Optimization) 훈련 데이터
ppo_jsonl =  path_join(repo_path, 'data_kochatgpt/kochatgpt_3_PPO.jsonl')
with open(ppo_jsonl, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

# 지도 미조정(Supervised Fine-Tuning, SFT)

## 모델 및 토큰 로드

In [12]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512)

In [13]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word

## SFT 데이터셋 클래스, 콜레이터 클래스 정의

In [14]:
from torch.utils.data import Dataset
import transformers, logging, copy
from typing import Optional, Dict, Sequence

# 프롬프트 형식 정의(전역 변수)
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

class SFT_dataset(Dataset):
    # 클래스 초기화 함수
    def __init__(self, data_path_1_SFT: str,
                 tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        # JSON 파일에서 데이터 로드
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        # 소스 데이터 생성
        sources = [
            PROMPT_DICT["prompt_input"].format_map(example) for example in list_data_dict]

        # 타겟 데이터 생성
        targets = [
            f"{example['completion']}{tokenizer.eos_token}" for example in list_data_dict]

        # 소스와 타겟 결합
        examples = [
            f"{source}{target}" for source, target in zip(sources, targets)
        ]

        # 토큰화 수행
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        # input_ids와 labels 준비
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100  # 소스 부분은 학습에서 제외

        data_dict = dict(input_ids=input_ids, labels=labels)

        # 클래스 인스턴스 변수 설정
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))

    # 토큰화 함수
    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )

    # 데이터셋 길이 반환 함수
    def __len__(self):
        return len(self.input_ids)

    # 특정 인덱스의 데이터 반환 함수
    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


In [15]:
from typing import Sequence, Dict
from torch.nn.utils.rnn import pad_sequence
from dataclasses import dataclass

@dataclass
class DataCollatorForSupervisedDataset(object):
    # 토크나이저 정의 (transformers 라이브러리에서 제공하는 PreTrainedTokenizer)
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        # 각 인스턴스에서 'input_ids'와 'labels'를 추출하여 튜플로 구성
        input_ids, labels = zip(*[(instance['input_ids'], instance['labels']) for instance in instances])

        # 'input_ids'와 'labels'를 패딩하여 동일한 길이로 맞춤
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100은 손실 계산 시 무시됨

        # attention_mask 생성: 패딩된 부분은 0, 그 외는 1
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

        # 결과 딕셔너리 반환
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

In [16]:
# 훈련셋과 콜레이터 인스턴스 생성
train_dataset = SFT_dataset(sft_jsonl, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [17]:
# 토큰 디코더
def decode_dataset_example(train_dataset, tokenizer, index=0):
    input_ids_example = train_dataset.input_ids[index]
    labels_example = train_dataset.labels[index]

    # -100 값을 제외하고 디코딩
    decoded_input = tokenizer.decode(input_ids_example, skip_special_tokens=True)
    decoded_labels = tokenizer.decode([t for t in labels_example if t != -100], skip_special_tokens=True)

    return decoded_input, decoded_labels

# 사용 예시
decoded_input, decoded_labels = decode_dataset_example(train_dataset, tokenizer, 1)
print("Decoded Input:\n", decoded_input)
print("")
print("Decoded Labels:\n", decoded_labels)

Decoded Input:
 ### Instruction(명령어):
쓰던 앱이 유료로 전환됐어

### Response(응답):'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.

Decoded Labels:
 '어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.


## 훈련 클래스 정의

In [32]:
from transformers import Trainer, TrainingArguments
import os

training_args = TrainingArguments(
    output_dir = path_join(repo_path, "test"),
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [33]:
# 훈련 진행
trainer.train()
model_path_SFT = path_join(repo_path, "output_1_SFT")
model.save_pretrained(model_path_SFT)

Step,Training Loss
500,2.867800
1000,2.789500
1500,2.694600


## 문장 생성

In [34]:
from transformers import pipeline
generator = pipeline('text-generation', model=model_path_SFT, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 정확한 답변을 드리기 어렵습니다. 하지만 일반적으로 불고기용 고기를 판매하는 가게나 음식점에 따라 다를 수 있으니, 해당 가게나 음식점의 공식 홈페이지나 전화번호를 확인해보시는 것을 추천드립니다. 감사합니다. you please provide m

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 41대 부통령직을 수행했습니다. J. W. Brown (Richard Johnson) J. W. B. Brown(Richard D. Nixon) J. W.B. Brown(Translation of the Capability) J. E. Brow

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 로스앤젤레스에 위치해 있습니다. Jefferson of Canada Operator 국제공항 (California Pacific Airportation Corporation) jefferson to Korean Capital Admin

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이기 때문에 미세먼지 문제에 대한 정보를 가지고 있지 않습니다. 하지만 미세먼지 문제가 심각하다는 것은 맞습니다. 미세먼지 농도가 높은 날에는 야외활동을 자제하는 것이 좋습니다. 또한 외출 시 마스크를 착용하거나 손세정제를 사용하는 것도 미세먼지를 줄이는 데 도움이


# 보상 모델(RM, Reward Model)

In [35]:
# 메모리 확보를 위해 캐시 삭제
torch.cuda.empty_cache()

## 사용자 정의 보상 모델 클래스

In [36]:
from chatgpt.models.base import RewardModel
from typing import Optional
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
import torch.nn as nn
from chatgpt.trainer.strategies import NaiveStrategy

class GPTRM_custom(RewardModel):

    # 초기화 함수
    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        # 사전 학습된 모델을 로드하는 경우
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        # 설정 객체를 사용하여 모델을 초기화하는 경우
        elif config is not None:
            model = GPT2Model(config)
        # 기본 설정으로 모델을 초기화하는 경우
        else:
            model = GPT2Model(GPT2Config())
        # 그래디언트 체크포인팅 활성화
        if checkpoint:
            model.gradient_checkpointing_enable()

        # 값(value) 헤드 추가
        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        # 사전 학습된 모델 정보 저장
        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained

    # 모델 저장 함수
    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

# GPT-2 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# 사용자 정의 모델 초기화 및 CUDA로 이동
with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()


In [37]:
# 모델 및 토크나이저 정의
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

In [38]:
# 훈련용 순위 데이터셋(ranking dataset) 생성

rm_jsonl = path_join(repo_path, 'data_kochatgpt/kochatgpt_2_RM.jsonl')
with open(rm_jsonl, "r", encoding='utf-8-sig') as json_file:
  list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [39]:
# 순위 데이터셋 섞은 후 훈련셋 생성
import random
from chatgpt.dataset import RewardDataset

random.seed(230319)
random.shuffle(total_data_ranking2chosen)

# 신속한 테스트를 위해 데이터 일부만 학습
train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 743.26it/s]


In [40]:
# 데이터셋 내용 확인
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [41]:
from chatgpt.trainer import RewardModelTrainer
from torch.optim import Adam

# 순위 모델 학습
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

trainer.fit(use_lora=0)

model_path_RM = path_join(repo_path, "output_2_RM")
model.save_pretrained(model_path_RM)

Train epoch: 100%|██████████| 1/1 [03:50<00:00, 230.70s/it]


In [42]:
# 학습 결과 확인

def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: 3.0


In [43]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: 3.1


In [44]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: 3.2


In [45]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: 3.3


In [46]:
# torch.cuda.empty_cache()
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [47]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained=model_path_SFT, lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=model_path_RM, lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())


In [48]:
# 옵티마이저 및 모델
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [49]:
# 데이터 로드
ppo_jsonl =  path_join(repo_path, 'data_kochatgpt/kochatgpt_3_PPO.jsonl')
with open(ppo_jsonl, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

# 토크나이저 정의
def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(tokenize_fn('It takes something more than intelligence to act intelligently.'))
len(list_prompt)

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


12000

In [50]:
# 별도 클래스 설계하여 학습 진행

trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,
                     train_batch_size=8,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

trainer.fit(list_prompt,
            num_episodes=10,
            max_timesteps=3,
            update_timesteps=3)

model_path_PPO = path_join(repo_path, "output_3_PPO")
model.save_pretrained(model_path_PPO)

Episode [10/10]: 100%|██████████| 3/3 [00:17<00:00,  5.73s/it]


# 최종 모델 테스트

In [52]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 AI 어시스턴트이므로 불고기용 고기를 실제로 먹을 수 없습니다. 해당 질문에 대해 더 자세히 설명해주시면 더 나은 답변을 드릴 수 있습니다. \n\n\n최근에는 국내산 한우 불고기용 고기를 찾는 사람들이 증가하고 있지만, 이 상황에서 직접 불고기용 고기를 찾아 먹을 수 있는 것은 아닙니다. 불고기용 고기를 찾아 먹을 수 있는 곳은 다양하지만, 인터넷에서 직접 주문하거나 직접 구워 먹을 수도 있습니다. \n\n하지만, 일부 불고기용 고기는 직접 구워먹는 경우도 있습니다. 예를 들어, 불고기용 고기를 직접 굽거나 포장할 수 있는 식당이나 식당 등이 있을 것입니다. \n\n하지만, 불고기용 고기를 찾아 먹을 수 있는 식당이 있다면 추가적인 정보를 제공해주시면 더 나은 답변을 드릴 수 있습니다. \n저는 인공지능 모델이기 때문에 이 질문에 대한 정확한 답변을 드릴 수 없습니다. \n\n감사합니다. q\n\n따라서 제가 직접 구워먹는 불고기용 고기를 찾아 먹을 수는 없습니다. \n\n

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 인공지능 언어모델이며, 41대 부통령직을 수행한 시기 및 관련 정보나 정보는 제공하지 않습니다. helpted phrasper (John Nixovict and Check)의 최신 논문 중 일부도 제공되지 않습니다. John Nixovict and Check이 41대 부통령직을 수행한 년도는 1996년이다. helpted phrasper, John Nixovict and Check, Nixovict and Check, Yes, Hally Yang (Morcen) 또는 Maunted Pork (Johns of Tore) 또는 Bernbel (Pathe) 입니다.宣信隆, Motherphs, Nixovict and Immotrain, John John Jamous

# 회고

학습용 코드를 코랩 환경에서 실행가능하도록 조정하는 데에 시간이 많이 걸렸다. 패키지 의존성이 아이펠 LMS 환경과 달라서 계속 에러가 발생했는데, 덕분에 어디서 의존성 문제가 발생했을 때 라이브러리 버전을 확인하고 그에 맞게 환경 설정을 하는 방법을 배웠다.

프로젝트는 학습 단계에서 완성한 코드의 세부 사항을 조정하는 것이었지만, 시간 부족으로 새로운 내용을 추가하지는 못했다.

다만 기반 모델을 바탕으로, 미세조정을 위한 다운스트림 태크스용 데이터셋을 준비하여 성능 조정 또는 용도 변경을 하는 과정에 대해서 이해할 수 있었고 관련된 주요 개념들에 대한 학습도 좋았다.

더 공부가 필요하겠다고 느낀 부분은, 기반 모델을 변경하거나 하이퍼파라미터 등을 조정해서 다른 결과를 만드는 것일 수도 있지만 근본적으로는 이미 기반모델이 머신러닝의 패러다임을 바꾸고 있기 때문에 상용 모델을 좀 더 효과적으로 개선하기 위한 작업 혹은 구체적인 태스크에 맞게 인프라를 구축하는 작업이라고 느낀다.

관련된 연구주제를 학습하고 궁극적으로 프로젝트도 이것과 관련된 것을 해보고 싶다.